In [2]:
import numpy as np
import pandas as pd

In [3]:
games = pd.read_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Raw\Games.csv")

C:\Users\himan\AppData\Local\Temp\ipykernel_11016\1153529159.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Raw\Games.csv")


In [4]:
games["awayTeam"] = games["awayteamCity"] + " " + games["awayteamName"]

In [5]:
games["homeTeam"] = games["hometeamCity"] + " " + games["hometeamName"]

In [6]:
games['Label'] = games['gameLabel'].fillna('') + ' ' + games['gameSubLabel'].fillna('')
games['Label'] = games['Label'].str.strip()


In [7]:
games["Label"].unique()

array(['', 'Emirates NBA Cup Championship',
       'Emirates NBA Cup West Semifinal',
       'Emirates NBA Cup East Semifinal',
       'Emirates NBA Cup West Quarterfinal',
       'Emirates NBA Cup East Quarterfinal',
       'Emirates NBA Cup West Group B', 'Emirates NBA Cup West Group A',
       'Emirates NBA Cup West Group C', 'Emirates NBA Cup East Group B',
       'Emirates NBA Cup East Group A', 'Emirates NBA Cup East Group C',
       'NBA Mexico City Game', 'Preseason', 'Preseason NBA China Game',
       'Preseason NBA Melbourne Game', 'Preseason NBA Abu Dhabi Game',
       'NBA Finals Game 7', 'NBA Finals Game 6', 'NBA Finals Game 5',
       'NBA Finals Game 4', 'NBA Finals Game 3', 'NBA Finals Game 2',
       'NBA Finals Game 1', 'East Conf. Finals Game 6',
       'East Conf. Finals Game 5', 'West Conf. Finals Game 5',
       'East Conf. Finals Game 4', 'West Conf. Finals Game 4',
       'East Conf. Finals Game 3', 'West Conf. Finals Game 3',
       'East Conf. Finals Game 2', 

In [ ]:
def classify_game_type(label):
    if label == '':
        return 'Regular'

    label = label.lower()

    # Play-In
    if 'play-in' in label:
        return 'PlayIn'

    # Playoffs
    playoff_keywords = [
        'first round',
        'semifinal',
        'semifinals',
        'conf. finals',
        'nba finals',
        'finals game'
    ]
    if any(k in label for k in playoff_keywords):
        return 'Playoffs'
    return None


games['game_type_clean'] = games['Label'].apply(classify_game_type)


In [9]:
games['game_type_clean'].value_counts(dropna=False)

game_type_clean
Regular     68573
Playoffs     3482
None          419
PlayIn         30
Name: count, dtype: int64

In [10]:
games['gameDateTimeEst'] = pd.to_datetime(
    games['gameDateTimeEst'],
    format='ISO8601',
    utc=True
)

In [11]:
games["gameDateTime"] = pd.to_datetime(games["gameDateTimeEst"]).dt.tz_localize(None)

In [12]:
games["season"] = games["gameDateTime"].apply(
    lambda x: x.year if x.month >= 10 else x.year-1
)

In [13]:
games = games[games['game_type_clean'].isin(['Regular', 'Playoffs', 'PlayIn'])]

In [14]:
games = games[games['season'] >= 2001]

In [15]:
games.columns

Index(['gameId', 'gameDateTimeEst', 'hometeamCity', 'hometeamName',
       'hometeamId', 'awayteamCity', 'awayteamName', 'awayteamId', 'homeScore',
       'awayScore', 'winner', 'gameType', 'attendance', 'arenaId', 'gameLabel',
       'gameSubLabel', 'seriesGameNumber', 'awayTeam', 'homeTeam', 'Label',
       'game_type_clean', 'gameDateTime', 'season'],
      dtype='object')

In [16]:
games = games.drop(columns=['hometeamCity', 'hometeamName', 'hometeamId', 'awayteamCity', 'awayteamName', 'gameLabel', 'gameSubLabel', 'Label', 'gameType', 'gameDateTimeEst'])


In [17]:
games.columns

Index(['gameId', 'awayteamId', 'homeScore', 'awayScore', 'winner',
       'attendance', 'arenaId', 'seriesGameNumber', 'awayTeam', 'homeTeam',
       'game_type_clean', 'gameDateTime', 'season'],
      dtype='object')

In [18]:
games['game_type_clean'].value_counts(dropna=False)

game_type_clean
Regular     30848
Playoffs     2004
PlayIn         30
Name: count, dtype: int64

In [19]:
games.shape

(32882, 13)

In [20]:
print(games['game_type_clean'].value_counts())
print(games['season'].min(), games['season'].max())
games['gameId'].nunique() == len(games)

game_type_clean
Regular     30848
Playoffs     2004
PlayIn         30
Name: count, dtype: int64
2001 2025


True

In [22]:
games.to_csv(r"C:\Users\himan\Downloads\DA\Project\NBA_Analytics\Data\Cleaned\games_clean.csv", index=False)